In [1]:
import os
from typing import Annotated, Sequence, TypedDict
from langchain_core.tools import Tool
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader, ArxivLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic

/Users/akashdeepsangwan/Desktop/Code/GenAI/AgenticAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [29]:
api_key= os.getenv("claudeAPI")

llm = ChatAnthropic(
    model = "claude-haiku-4-5-20251001",
    temperature = 0.7,
    api_key = api_key
)

Embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
urls=[
    "https://docs.langchain.com/oss/python/langgraph/overview",
    "https://docs.langchain.com/oss/python/langgraph/workflows-agents",
    "https://docs.langchain.com/oss/python/langgraph/graph-api#map-reduce-and-the-send-api"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

chunks = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100).split_documents(docs_list) # each element is a doc
vector_store = FAISS.from_documents(chunks,Embeddings)
retriever = vector_store.as_retriever()


In [58]:
result =retriever.invoke("what is langgraph ?")
print(result[0].page_content)

​Acknowledgements
LangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.


In [ ]:
from langchain_core.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retriever_vector_db_blog",
    "Search and rin information about LangGraph"
)


### Tool is working

In [ ]:
retriever_tool.invoke("What is langGraph?")

'\u200bAcknowledgements\nLangGraph is inspired by Pregel and Apache Beam. The public interface draws inspiration from NetworkX. LangGraph is built by LangChain Inc, the creators of LangChain, but can be used without LangChain.\n\nAt its core, LangGraph models agent workflows as graphs. You define the behavior of your agents using three key components:\n\nprint(f"Step: {metadata[\'langgraph_step\']}")\n    print(f"Node: {metadata[\'langgraph_node\']}")\n    print(f"Triggers: {metadata[\'langgraph_triggers\']}")\n    print(f"Path: {metadata[\'langgraph_path\']}")\n    print(f"Checkpoint NS: {metadata[\'langgraph_checkpoint_ns\']}")\n\n    return state\n\n\u200bVisualization\nIt’s often nice to be able to visualize graphs, especially as they get more complex. LangGraph comes with several built-in ways to visualize graphs. See this how-to guide for more info.\n\nEdit this page on GitHub or file an issue.\nConnect these docs to Claude, VSCode, and more via MCP for real-time answers.Was this

In [68]:
langchain_url  = [
    "https://docs.langchain.com/oss/python/langchain/overview",
    "https://docs.langchain.com/oss/python/langchain/rag",
]

web_doc_load = [WebBaseLoader(url).load() for url in langchain_url]
langchain_docs = [doc for sublist in web_doc_load for doc in sublist]
chunks_lang = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap = 100).split_documents(langchain_docs)
vb = FAISS.from_documents(chunks_lang,Embeddings)
langchain_retriever = vb.as_retriever()


In [ ]:
langchain_tool  = create_retriever_tool(
    langchain_retriever,
    "retriever_vector_langchain_blog",
    "Search and run information about LangChain"
)


In [ ]:
tools = [retriever_tool, langchain_tool]